In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spam-text-message-classification/SPAM text message 20170820 - Data.csv


In [2]:
data = pd.read_csv('/kaggle/input/spam-text-message-classification/SPAM text message 20170820 - Data.csv')
data = data.replace(to_replace="ham", value=0)
data = data.replace(to_replace="spam", value=1)
data.head()

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
# split data into train and test data
import tensorflow as tf

sentences = data['Message'].tolist()
labels = data['Category'].tolist()


#separate sentences and labels into training and test
training_size = int(len(sentences) * 0.8)
train_sentences = sentences[0:training_size]
test_sentences = sentences[training_size:]
train_labels = labels[0:training_size]
test_labels = labels[training_size:]

#make labels into numpy arrays for use with the network later
train_labels_arr = np.array(train_labels)
test_labels_arr = np.array(test_labels)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
vocab_size = 1000
embedding_dimension = 16
max_length = 100
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

In [5]:
sequences = tokenizer.texts_to_sequences(train_sentences)

#introduce padding to make sure all sentences have the same length for purposes of analysis by the RNN
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


In [6]:
#the recurrent neural network (RNN) is built and trained across 20 epochs, 
#the input layer is comprised of an embedding layer which represents the sentences with 
#dense vector representation

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dimension, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 16)           16000     
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 6)                 9606      
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 25,613
Trainable params: 25,613
Non-trainable params: 0
_________________________________________________________________


In [7]:
num_epochs = 20
history=model.fit(padded, train_labels_arr, epochs=num_epochs, validation_data=(test_padded,test_labels_arr))

Epoch 1/20
140/140 [==============================] - 2s 4ms/step - loss: 0.3023 - accuracy: 0.8622 - val_loss: 0.1727 - val_accuracy: 0.8700
Epoch 2/20
140/140 [==============================] - 0s 3ms/step - loss: 0.1419 - accuracy: 0.9531 - val_loss: 0.0794 - val_accuracy: 0.9767
Epoch 3/20
140/140 [==============================] - 0s 3ms/step - loss: 0.0576 - accuracy: 0.9845 - val_loss: 0.0541 - val_accuracy: 0.9830
Epoch 4/20
140/140 [==============================] - 0s 3ms/step - loss: 0.0391 - accuracy: 0.9895 - val_loss: 0.0514 - val_accuracy: 0.9839
Epoch 5/20
140/140 [==============================] - 0s 3ms/step - loss: 0.0299 - accuracy: 0.9917 - val_loss: 0.0483 - val_accuracy: 0.9830
Epoch 6/20
140/140 [==============================] - 0s 3ms/step - loss: 0.0220 - accuracy: 0.9948 - val_loss: 0.0478 - val_accuracy: 0.9865
Epoch 7/20
140/140 [==============================] - 0s 3ms/step - loss: 0.0174 - accuracy: 0.9957 - val_loss: 0.0533 - val_accuracy: 0.9865
Epoch 